In [24]:
# import data from drive

from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/Data.zip"
!unzip -q "/content/gdrive/My Drive/Covid_model.zip"

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
replace Data/Covid/01E392EE-69F9-4E33-BFCE-E5C968654078.jpeg? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
replace Covid_model.h5? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


In [ ]:
'''

project : Covid-19-detection-using-Xray
Author : @kanishksh4rma

'''

'\n\nproject : Covid-19-detection-using-Xray\nAuthor : @kanishksh4rma\n\n'

In [ ]:
from imutils import paths
import matplotlib.pyplot as plt
import argparse
import os

In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras.layers import AveragePooling2D, Dropout,Flatten,Dense,Input
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

In [ ]:
dataset = "/content/Data"   # path to the dataset
args={}
args["dataset"]=dataset

In [ ]:
import numpy as np
import cv2
img_paths = list(paths.list_images(args["dataset"]))  #image paths
data = []
labels = []
for path in img_paths:
    label = path.split(os.path.sep)[-2]   #split the image paths
    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) #Convert images into RGB Channel
                                                   # Though it isnt necessary for X-ray images
    
    image = cv2.resize(image, (224, 224))  #Resizing the images
    data.append(image)
    labels.append(label)
data = np.array(data) / 255.0
labels = np.array(labels)

In [ ]:
Data_Dir = "Data//"

In [ ]:
Cimages = os.listdir(Data_Dir+"Covid")
Nimages = os.listdir(Data_Dir+"Normal")

In [ ]:
import matplotlib.pyplot as plt
import cv2
import skimage
from skimage.transform import resize
import numpy as np

# plot some of the X-rays
def plotter(i):
    normal = cv2.imread(Data_Dir+"Normal//"+Nimages[i])
    normal = skimage.transform.resize(normal, (150, 150, 3))
    coronavirus = cv2.imread(Data_Dir+"Covid//"+Cimages[i])
    coronavirus = skimage.transform.resize(coronavirus, (150, 150, 3) , mode = 'reflect')
    pair = np.concatenate((normal, coronavirus), axis=1)
    print("Normal Chest X-ray Vs Covid-19 Chest X-ray")
    plt.figure(figsize=(10,5))
    plt.imshow(pair)
    plt.show()
for i in range(0,5):
    plotter(i)


In [ ]:
LB = LabelBinarizer()  
#Initialize label binarizer
labels = LB.fit_transform(labels)
labels = to_categorical(labels)

In [ ]:
# test train split
(X_train, X_test, Y_train, Y_test) = train_test_split(data, labels,
    test_size=0.20, stratify=labels, random_state=42)

In [ ]:
#rotate images to create more data
trainAug = ImageDataGenerator(
    rotation_range=15,
    fill_mode="nearest")

In [ ]:
bModel = VGG16(weights="imagenet", include_top=False,input_tensor=Input(shape=(224, 224, 3)))  #base_Model
hModel = bModel.output #head_Model
hModel = AveragePooling2D(pool_size=(4, 4))(hModel)
hModel = Flatten(name="flatten")(hModel)
hModel = Dense(64, activation="relu")(hModel)
hModel = Dropout(0.5)(hModel)
hModel = Dense(2, activation="softmax")(hModel)
model = Model(inputs=bModel.input, outputs=hModel)

for layer in bModel.layers:
    layer.trainable = False

In [ ]:
X_train.shape,X_test.shape,Y_train.shape,Y_test.shape

In [ ]:
INIT_LR = 1e-3
EPOCHS = 10
BS = 8

In [ ]:
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt,metrics=["accuracy"])

R = model.fit_generator(
    trainAug.flow(X_train, Y_train, batch_size=BS),
    steps_per_epoch=len(X_train) // BS,
    validation_data=(X_test, Y_test),
    validation_steps=len(X_test) // BS,
    epochs=EPOCHS)

In [ ]:

# Let's test on some random test data

L = 6
W = 5
fig, axes = plt.subplots(L, W, figsize = (12, 12))
axes = axes.ravel()
y_pred = model.predict(X_test, batch_size=BS)
for i in np.arange(0,L*W):
    axes[i].imshow(X_test[i])
    axes[i].set_title('Prediction = {}\n True = {}'.format(y_pred.argmax(axis=1)[i], Y_test.argmax(axis=1)[i]))
    axes[i].axis('off')

plt.subplots_adjust(wspace = 0.5, hspace=0.5)  
print('0 : Affected by COVID19\n1 : Healthy Person') 

Wow! All every prediction are correct.

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test, batch_size=BS)
y_pred = np.argmax(y_pred, axis=1)
print(classification_report(Y_test.argmax(axis=1), y_pred,target_names=LB.classes_))

In [ ]:
from sklearn.metrics import accuracy_score
print('Accuracy score : ',accuracy_score(Y_test.argmax(axis=1),y_pred)*100,'%')

In [ ]:
from sklearn.metrics import confusion_matrix

# check for Sensitivity & Specificity

cm = confusion_matrix(Y_test.argmax(axis=1), y_pred)
total = sum(sum(cm))
acc = (cm[0, 0] + cm[1, 1]) / total
sensitivity = cm[0, 0] / (cm[0, 0] + cm[0, 1])
specificity = cm[1, 1] / (cm[1, 0] + cm[1, 1])
print(cm)
print("acc: {:.4f}".format(acc))
print("sensitivity: {:.4f}".format(sensitivity))
print("specificity: {:.4f}".format(specificity))

In [ ]:
# plot the loss
plt.plot(R.history['loss'], label='train loss')
plt.plot(R.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# plot the accuracy
plt.plot(R.history['accuracy'], label='train acc')
plt.plot(R.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('LossVal_acc')

In [ ]:
model.save('Covid_model.h5')

In [ ]:
from flask import jsonify
from keras.preprocessing import image
from keras.models import load_model
print('making model ')
from google.colab import files
image1 = files.upload()
image_list = list(image1.keys())
image1 = image_list[0]
print('Image : ',image1)
print('=========================')
print('File uploaded successfully!!!')
print('=========================')

making model 


Saving WhatsApp Image 2020-11-20 at 18.52.20.jpeg to WhatsApp Image 2020-11-20 at 18.52.20 (1).jpeg
Image :  WhatsApp Image 2020-11-20 at 18.52.20.jpeg
File uploaded successfully!!!


In [ ]:
import numpy as np
from flask import jsonify
from keras.preprocessing import image
from keras.models import load_model
print('fetching results...')
new_model = load_model('Covid_model.h5')
test_image = image.load_img(image1,target_size=(224,224))
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)

result = new_model.predict(test_image)
y_pred = np.argmax(result, axis=1)
print('Model prediction: ',result)

#test purpose statement
#if result[0][0]==1.0 and result[0][1]>=1.8687282e-25 or result[0][1]==1.0 and result[0][0]>=1.8687282e-25:

# make it main if iff want to upload xrays only

'''if result[0][0]==1.0 and result[0][1]<=0.3 or result[0][1]==1.0 and result[0][0]<=0.3:
  prediction = "Please upload x ray only"'''

if result[0][0]<result[0][1] and result[0][0]>4.226988e-15:
  prediction = 'Patient is affected with COVID-19'
  #return 1
else:
  prediction = 'Patient is Healthy'
  #return 0


print('YPred',y_pred)
print('===================================')
print(prediction)
print('===================================')


fetching results...
Model prediction:  [[1.000000e+00 5.652805e-13]]
YPred [0]
Please upload x ray only
